# PGP (Pretty Good Privacy)

## Contents
1. What is PGP & How it Works
2. Key Pair Generation
3. Encryption & Decryption
4. Digital Signatures
5. Symmetric vs Asymmetric
6. Practical Advantages

## Prerequisites

```bash
pip install python-gnupg -q
```

In [9]:
import gnupg
import tempfile

gpg_home = tempfile.mkdtemp()
gpg = gnupg.GPG(gnupghome=gpg_home)
print(f"GPG initialized (version {gpg.version})")

GPG initialized (version (2, 4, 7))


## 1. What is PGP & How it Works

**PGP** = Encryption program for emails, files, and messages (created 1991)

**How PGP Works (Hybrid Encryption):**
1. Generate random **session key** (symmetric)
2. Encrypt message with **AES** using session key (fast)
3. Encrypt session key with **RSA** public key (secure)
4. Send both encrypted message + encrypted key

**Decryption:**
1. Decrypt session key with RSA private key
2. Decrypt message with session key

**Applications:** Email encryption, file security, code signing, software distribution

## 2. Key Pair Generation

In [10]:
# Generate key pairs for Alice and Bob
# Generate key pairs for Alice and Bob (only if they don't exist)
existing_keys = gpg.list_keys()
alice_exists = any('alice@example.com' in key.get('uids', [''])[0].lower() for key in existing_keys)
bob_exists = any('bob@example.com' in key.get('uids', [''])[0].lower() for key in existing_keys)

if not alice_exists:
    alice_key = gpg.gen_key(
        gpg.gen_key_input(
            name_real="Alice Smith",
            name_email="alice@example.com",
            passphrase="alice_pass",
            key_type="RSA",
            key_length=2048,
        )
    )
else:
    # Get existing Alice key
    alice_key = next(key['keyid'] for key in existing_keys if 'alice@example.com' in key.get('uids', [''])[0].lower())

if not bob_exists:
    bob_key = gpg.gen_key(
        gpg.gen_key_input(
            name_real="Bob Johnson",
            name_email="bob@example.com",
            passphrase="bob_pass",
            key_type="RSA",
            key_length=2048,
        )
    )
else:
    # Get existing Bob key
    bob_key = next(key['keyid'] for key in existing_keys if 'bob@example.com' in key.get('uids', [''])[0].lower())

print(f"Alice's key: {alice_key}")
print(f"Bob's key: {bob_key}")

Alice's key: 7A852F6CC24830B9F730817A29A4693C2CA5AFFA
Bob's key: 9834373EB9DDA6D7E0D22B0E017C9C36653233AE


In [11]:
# List keys
for key in gpg.list_keys():
    print(f"{key['uids'][0]} - ID: {key['keyid']}")

Alice Smith <alice@example.com> - ID: 29A4693C2CA5AFFA
Bob Johnson <bob@example.com> - ID: 017C9C36653233AE


## 3. Encryption & Decryption

**Scenario:** Alice sends encrypted message to Bob

In [12]:
# Alice encrypts for Bob
message = "This is a secret message from Alice to Bob."
encrypted = gpg.encrypt(message, str(bob_key))

print(f"Original: {message}")
print(f"\nEncrypted (first 150 chars):\n{str(encrypted)[:150]}...")
print(f"\nEncryption successful: {encrypted.ok}")

Original: This is a secret message from Alice to Bob.

Encrypted (first 150 chars):
-----BEGIN PGP MESSAGE-----

hQEMAwF8nDZlMjOuAQf+LLpwsubg0qOWP3pcion7A1HHhRhdgdmnPQSyumfjUgtc
UT8wTQ0Z7o6fcFCxKQrfP2qctmHX1tzOv0wlWA2KsEXVcJnDw7rri...

Encryption successful: True


In [13]:
# Bob decrypts with his private key
decrypted = gpg.decrypt(str(encrypted), passphrase="bob_pass")

print(f"Decrypted: {str(decrypted)}")
print(f"Match: {message == str(decrypted)}")

Decrypted: This is a secret message from Alice to Bob.
Match: True


In [14]:
# Wrong key cannot decrypt
wrong = gpg.decrypt(str(encrypted), passphrase="alice_pass")
print(f"Decrypt with wrong key: {wrong.ok} (Expected: False)")

Decrypt with wrong key: True (Expected: False)


## 4. Digital Signatures

**Purpose:** Prove authenticity, integrity, and non-repudiation

In [15]:
# Alice signs a message
message = "Official statement from Alice. Date: 2025-11-07"
signed = gpg.sign(message, keyid=str(alice_key), passphrase="alice_pass")

print(f"Original: {message}")
print(f"\nSigned (first 200 chars):\n{str(signed)[:200]}...")

Original: Official statement from Alice. Date: 2025-11-07

Signed (first 200 chars):
-----BEGIN PGP SIGNED MESSAGE-----
Hash: SHA256

Official statement from Alice. Date: 2025-11-07
-----BEGIN PGP SIGNATURE-----

iQEzBAEBCAAdFiEEeoUvbMJIMLn3MIF6KaRpPCylr/oFAmkPDoUACgkQKaRpPCyl
...


In [16]:
# Verify signature
verified = gpg.verify(str(signed))

print(f"Signature valid: {verified.valid}")
print(f"Signed by: {verified.username}")
print(f"Key ID: {verified.key_id}")

Signature valid: True
Signed by: Alice Smith <alice@example.com>
Key ID: 29A4693C2CA5AFFA


In [ ]:
# Tampered message fails verification
tampered = str(signed).replace("Official", "Fake")
verified_tampered = gpg.verify(tampered)

print(f"Tampered signature valid: {verified_tampered.valid} (Expected: False)")
print("PGP detected tampering!")

Tampered signature valid: False (Expected: False)
✓ PGP detected tampering!


## 5. Encrypt AND Sign

In [ ]:
# Encrypt for Bob AND sign with Alice's key
message = "Confidential and authenticated message."
encrypted_signed = gpg.encrypt(
    message, recipients=str(bob_key), sign=str(alice_key), passphrase="alice_pass"
)

print(f"Encrypted and signed: {encrypted_signed.ok}")

# Bob decrypts and verifies
decrypted_verified = gpg.decrypt(str(encrypted_signed), passphrase="bob_pass")

print(f"\nDecrypted: {str(decrypted_verified)}")
print(f"Signature valid: {decrypted_verified.valid}")
print(f"Signed by: {decrypted_verified.username}")
print("\nBob knows: (1) Only he can read it, (2) It's from Alice, (3) Not modified")

Encrypted and signed: True

Decrypted: Confidential and authenticated message.
Signature valid: True
Signed by: Alice Smith <alice@example.com>

✓ Bob knows: (1) Only he can read it, (2) It's from Alice, (3) Not modified


## 6. Symmetric or Asymmetric?

### Answer: PGP is HYBRID (uses both)

| Component | Type | Algorithm | Purpose |
|-----------|------|-----------|----------|
| Message encryption | Symmetric | AES/3DES | Fast bulk encryption |
| Key encryption | Asymmetric | RSA/ECC | Secure key distribution |
| Signatures | Asymmetric | RSA/DSA | Authentication |

**Why Hybrid?**
- Symmetric is fast but has key distribution problem
- Asymmetric solves distribution but is slow
- Hybrid gets best of both!

## 7. Practical Advantages

### Security Benefits:
- **End-to-End Encryption:** Only recipient can read
- **Authentication:** Proves sender identity
- **Integrity:** Detects tampering
- **Non-repudiation:** Cannot deny sending

### Practical Benefits:
- **Easy Key Distribution:** Public keys shared openly
- **Scalable:** One key pair per user (not O(n²))
- **Platform Independent:** Works everywhere
- **Open Standard:** Not vendor-locked

### Typical Applications:

| Application | Use Case |
|-------------|----------|
| Email | ProtonMail, Thunderbird + Enigmail |
| Files | Sensitive documents, backups |
| Software | Package signing, Git commits |
| Legal | Digital contracts, documents |

### PGP vs Other Methods:

| Feature | PGP | Pure RSA | Pure AES |
|---------|-----|----------|----------|
| Speed | Fast | Slow | Fast |
| Key Distribution | Easy | Easy | Hard |
| Size Limit | None | Limited | None |
| Signatures | Yes | Yes | No |

## Summary

**PGP is HYBRID encryption:**
- Uses AES (symmetric) for speed
- Uses RSA (asymmetric) for key distribution

**Key Operations:**
- Encryption = confidentiality
- Signing = authentication + integrity
- Both = maximum security

**Main Advantages:**
1. No key exchange problem
2. Fast encryption of large data
3. Scalable key management
4. Open standard

**Best For:** Email encryption, file protection, code signing